In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())

    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/1D FODE/atanh'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1]).reshape(-1,1)
n_value = np.array([1.0,3.0,5.0,8.0,10.0]).reshape(-1,1)

LR_tune,N_value = np.meshgrid(lr_tune,n_value)

LR_tune = LR_tune.flatten('F').reshape(-1,1)
N_value = N_value.flatten('F').reshape(-1,1)

lrn_tune = np.hstack((LR_tune,N_value))

In [6]:
def true_2D_4(xt): #True function for 2D_4 Heat Transfer in a rod x \in [0,1] t \in [0,0.1]
    term1 = 4*u0/np.pi
    
    resol_n = 10000
    
    x = xt[:,0].reshape(-1,1)
    t = xt[:,1].reshape(-1,1)

    u = np.zeros((np.shape(xt)[0],1))
    
    for i in range(resol_n):
        j = 2*i-1
        term2 = np.sin(j*np.pi*x)/j
        term3 = np.exp(-1*np.square(j*np.pi)*t)
        
        u = u + term2*term3
        
    u = term1*u
    
    return u

In [7]:
u0 = 50.0
loss_thresh = 0.1
label = "inv_HT_atanh"
    
x_ll = np.array(0.0)
x_ul = np.array(1.0)

x = np.linspace(x_ll,x_ul,100).reshape(-1,1)
t = np.linspace(0,0.1,100).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

u_true = true_2D_4(xt)
u_true_norm = np.linalg.norm(u_true,2)


lb_xt = xt[0]
ub_xt = xt[-1]

xt_test_tensor = torch.from_numpy(xt).float().to(device)

In [8]:
def trainingdata(N_f,N_train,seed):
    
    np.random.seed(seed)
    
    #X_Train
    np.random.seed(seed)
    x_train = np.random.uniform(x_ll,x_ul,(N_train,1))
    t_train = np.random.uniform(0,0.1,(N_train,1))
    
    xt_train = np.hstack((x_train,t_train))
    u_train = true_2D_4(xt_train)
    

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)

    samples = sampling(N_f)
    
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll)) # append training points to collocation points 

    return xt_coll, xt_train, u_train

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers,n_val):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Tanh()

     
        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        self.iter = 0
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
        
        
        self.alpha = Parameter(torch.ones((50,len(layers)-2)))
        self.alpha.requiresGrad = True

        
        self.n = torch.tensor(n_val)
        
    
        self.lambda1 = Parameter(torch.tensor(0.0))
        self.lambda1.requiresGrad = True
        

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = self.activation(self.n*self.alpha[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    
    def loss_PDE(self, xt_coll,f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        u = self.forward(g) 
        
        u_x_t = autograd.grad(u,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        u_xx_tt = autograd.grad(u_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        du_dx = u_x_t[:,[0]]
        
        d2u_dx2 = u_xx_tt[:,[0]]
                
        du_dt = u_x_t[:,[1]]
        
        f = du_dt - self.lambda1*d2u_dx2
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_coll,f_hat, xt_train, u_train):

        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_train = self.loss_function(self.forward(xt_train),u_train)
        
        loss_val = loss_f + loss_train
        
        #print(self.iter,"train_loss",loss_train.cpu().detach().numpy(),"F Loss",(loss_f).cpu().detach().numpy())
        return loss_val
     
    'callable for optimizer'                                    
    
    def test(self):
        u_pred = self.forward(xt_test_tensor)
        u_pred = u_pred.cpu().detach().numpy()
   
        return u_pred

    def test_loss(self):
        u_pred = self.test()
               
        test_mse = np.mean(np.square(u_pred.reshape(-1,1) - u_true.reshape(-1,1)))
        test_re = np.linalg.norm(u_pred.reshape(-1,1) - u_true.reshape(-1,1),2)/u_true_norm
        
        return test_mse, test_re 

In [10]:
def train_step(xt_coll,f_hat, xt_train, u_train,seed):    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_coll,f_hat, xt_train, u_train)
        loss.backward()
        
        return loss

    optimizer.step(closure)

In [11]:
def data_update(loss_np):
    train_loss.append(loss_np)
    alpha_val.append(PINN.alpha.cpu().detach().numpy())
    lambda1_val.append(PINN.lambda1.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [12]:
def train_model(max_iter,rep): 
    print(rep) 
    torch.manual_seed(rep*11)
    start_time = time.time() 
    thresh_flag = 0
    
    xt_coll, xt_train, u_train = trainingdata(N_f,N_train,123)
    
    xt_coll = torch.from_numpy(xt_coll).float().to(device)
    xt_train = torch.from_numpy(xt_train).float().to(device)
    u_train = torch.from_numpy(u_train).float().to(device)
    
    
    f_hat = torch.zeros(xt_coll.shape[0],1).to(device)
    nan_flag = 0
    
    for i in range(max_iter):
        train_step(xt_coll,f_hat, xt_train, u_train,i)

        loss_np = PINN.loss(xt_coll,f_hat, xt_train, u_train).cpu().detach().numpy()
        
        if(thresh_flag == 0):
            if(loss_np < loss_thresh):
                time_threshold[rep] = time.time() - start_time
                epoch_threshold[rep] = i+1            
                thresh_flag = 1       
        data_update(loss_np)
        print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1],"Lambda1",lambda1_val[-1])

        if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break
    
    elapsed_time[rep] = time.time() - start_time  
    print('Training time: %.2f' % (elapsed_time[rep]))

In [13]:

max_reps = 10 #10
max_iter = 200#75

train_loss_full = []
test_mse_full = []
test_re_full = []
alpha_full = []

lambda1_full = []
elapsed_time= np.zeros((max_reps,1))

time_threshold = np.empty((max_reps,1))
time_threshold[:] = np.nan
epoch_threshold = max_iter*np.ones((max_reps,1))

n_val = 3.0

for reps in range(max_reps):
    print(label)
    'Generate Training data'
    print(reps)
    torch.manual_seed(reps*36)

    train_loss = []
    test_mse_loss = []
    test_re_loss = []   
    alpha_val = []

    lambda1_val = []

    N_f = 50000 #Total number of collocation points 
    N_train = 5000

    layers = np.array([2,50,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

    PINN = Sequentialmodel(layers,n_val)

    PINN.to(device)

    'Neural Network Summary'
    print(PINN)

    params = list(PINN.parameters())

    optimizer = torch.optim.LBFGS(PINN.parameters(), lr=0.25, 
                              max_iter = 20, 
                              max_eval = 30, 
                              tolerance_grad = 1e-8, 
                              tolerance_change = 1e-8, 
                              history_size = 100, 
                              line_search_fn = 'strong_wolfe')


    nan_flag = train_model(max_iter,reps)

    torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
    train_loss_full.append(train_loss)
    test_mse_full.append(test_mse_loss)
    test_re_full.append(test_re_loss)
    #elapsed_time[reps] = time.time() - start_time
    alpha_full.append(alpha_val)

    lambda1_full.append(lambda1_val)

    if(nan_flag == 1):
        nan_tune.append(tune_reps)
        break

    print('Training time: %.2f' % (elapsed_time[reps]))

mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "alpha": alpha_full,"lambda1": lambda1_full, "label": label,"Thresh Time": time_threshold,"Thresh epoch": epoch_threshold}
savemat(label+'.mat', mdic) 


inv_HT_atanh
0
Sequentialmodel(
  (activation): Tanh()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=50, bias=True)
    (10): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 21311.184 Test MSE 3572.210103913615 Test RE 1.0061382689139065 Lambda1 -0.00090391986
1 Train Loss 15290.309 Test MSE 3568.5830097285166 Test RE 1.0056273408069494 Lambda1 -0.00032179378
2 Train Loss 11761.734 

75 Train Loss 2789.238 Test MSE 2712.8961262806242 Test RE 0.8768105230279057 Lambda1 1.796106e-05
76 Train Loss 2737.8342 Test MSE 2668.9465506038737 Test RE 0.8696792513743353 Lambda1 4.8907394e-05
77 Train Loss 2695.6768 Test MSE 2604.499438095583 Test RE 0.8591150056567345 Lambda1 3.7260146e-05
78 Train Loss 2658.1104 Test MSE 2571.0585745498556 Test RE 0.853581818853269 Lambda1 4.2630607e-05
79 Train Loss 2629.4937 Test MSE 2552.61730253731 Test RE 0.8505150932034518 Lambda1 4.238512e-05
80 Train Loss 2563.8464 Test MSE 2494.4467048862016 Test RE 0.8407682157856204 Lambda1 1.876501e-05
81 Train Loss 2537.835 Test MSE 2470.9083971737323 Test RE 0.8367919493951236 Lambda1 1.9411042e-05
82 Train Loss 2504.1702 Test MSE 2443.738505853805 Test RE 0.8321785869846279 Lambda1 6.3229427e-06
83 Train Loss 2472.0928 Test MSE 2395.5880075550276 Test RE 0.8239393333813668 Lambda1 -6.321793e-07
84 Train Loss 2433.8313 Test MSE 2355.723161621043 Test RE 0.817055008650791 Lambda1 -1.6302996e-05
8

157 Train Loss 683.56604 Test MSE 693.9449211343497 Test RE 0.4434571600743195 Lambda1 -1.7467232e-06
158 Train Loss 683.4444 Test MSE 694.0963389165795 Test RE 0.4435055382922342 Lambda1 -1.986197e-06
159 Train Loss 683.2514 Test MSE 694.1179501941036 Test RE 0.44351244269881895 Lambda1 -1.4037613e-06
160 Train Loss 682.96265 Test MSE 694.0508087852877 Test RE 0.4434909918990249 Lambda1 -1.7444244e-06
161 Train Loss 681.1352 Test MSE 692.9062383738973 Test RE 0.44312515692776167 Lambda1 -1.0887761e-06
162 Train Loss 679.06805 Test MSE 692.2597789731434 Test RE 0.4429183978788467 Lambda1 6.6713716e-07
163 Train Loss 677.7757 Test MSE 691.7860047895665 Test RE 0.4427668079522294 Lambda1 -9.667626e-07
164 Train Loss 676.8005 Test MSE 689.0913205497229 Test RE 0.44190362130669464 Lambda1 -1.399506e-06
165 Train Loss 675.7735 Test MSE 688.4456929666986 Test RE 0.4416965572955642 Lambda1 -1.5368855e-06
166 Train Loss 674.1864 Test MSE 688.3649064784679 Test RE 0.44167064082824303 Lambda1 6.

31 Train Loss 639.3216 Test MSE 662.5284994257556 Test RE 0.4333027576210353 Lambda1 1.0380403e-05
32 Train Loss 638.38165 Test MSE 661.0892143815609 Test RE 0.43283184561366567 Lambda1 1.5800041e-06
33 Train Loss 637.8712 Test MSE 660.8574087070858 Test RE 0.43275595443854326 Lambda1 -4.3037944e-06
34 Train Loss 635.0114 Test MSE 656.950651723238 Test RE 0.4314749077230557 Lambda1 -8.2740735e-07
35 Train Loss 633.7749 Test MSE 656.986317822409 Test RE 0.4314866200319977 Lambda1 -1.2946134e-07
36 Train Loss 633.47876 Test MSE 656.841804396664 Test RE 0.43143916163194507 Lambda1 -9.830577e-07
37 Train Loss 633.34204 Test MSE 656.6708873519689 Test RE 0.4313830255093655 Lambda1 -5.897661e-08
38 Train Loss 632.26746 Test MSE 655.3129504473169 Test RE 0.43093676384613033 Lambda1 1.6109124e-06
39 Train Loss 630.9653 Test MSE 653.7611829656391 Test RE 0.43042623677599917 Lambda1 7.229258e-06
40 Train Loss 630.09033 Test MSE 653.6239603610186 Test RE 0.4303810617866269 Lambda1 -5.2426503e-06


114 Train Loss 623.7998 Test MSE 647.8734380561974 Test RE 0.4284836531514184 Lambda1 4.9628557e-08
115 Train Loss 623.7998 Test MSE 647.8734380561974 Test RE 0.4284836531514184 Lambda1 4.9628557e-08
116 Train Loss 623.7998 Test MSE 647.8734380561974 Test RE 0.4284836531514184 Lambda1 4.9628557e-08
117 Train Loss 623.7998 Test MSE 647.8734380561974 Test RE 0.4284836531514184 Lambda1 4.9628557e-08
118 Train Loss 623.7998 Test MSE 647.8734380561974 Test RE 0.4284836531514184 Lambda1 4.9628557e-08
119 Train Loss 623.7998 Test MSE 647.8734380561974 Test RE 0.4284836531514184 Lambda1 4.9628557e-08
120 Train Loss 623.7998 Test MSE 647.8734380561974 Test RE 0.4284836531514184 Lambda1 4.9628557e-08
121 Train Loss 623.7998 Test MSE 647.8734380561974 Test RE 0.4284836531514184 Lambda1 4.9628557e-08
122 Train Loss 623.7998 Test MSE 647.8734380561974 Test RE 0.4284836531514184 Lambda1 4.9628557e-08
123 Train Loss 623.7998 Test MSE 647.8734380561974 Test RE 0.4284836531514184 Lambda1 4.9628557e-08


...

57 Train Loss 454.03018 Test MSE 463.14071497862284 Test RE 0.3622810781313766 Lambda1 3.597703e-07
58 Train Loss 453.28424 Test MSE 462.37043551070633 Test RE 0.3619796861425002 Lambda1 1.0097416e-06
59 Train Loss 451.04398 Test MSE 461.3473706939118 Test RE 0.36157899685541545 Lambda1 1.1620562e-06
60 Train Loss 449.89795 Test MSE 459.1582055010071 Test RE 0.36072010239367813 Lambda1 1.0561316e-06
61 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
62 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
63 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
64 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
65 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
66 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 La

137 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
138 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
139 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
140 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
141 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
142 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
143 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
144 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
145 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.35896465270522776 Lambda1 -2.8106235e-06
146 Train Loss 446.93036 Test MSE 454.70007939630665 Test RE 0.3

9 Train Loss 855.862 Test MSE 858.8127624789797 Test RE 0.49333101011318115 Lambda1 -0.12787277
10 Train Loss 854.6865 Test MSE 858.026230129249 Test RE 0.4931050530145129 Lambda1 -0.12931909
11 Train Loss 854.5149 Test MSE 857.7211326661173 Test RE 0.4930173759297886 Lambda1 -0.10515917
12 Train Loss 854.4164 Test MSE 857.5117400847188 Test RE 0.49295719291784507 Lambda1 -0.08866099
13 Train Loss 853.8497 Test MSE 856.5657043897595 Test RE 0.4926851944282462 Lambda1 -0.065564156
14 Train Loss 853.14453 Test MSE 854.7881929581521 Test RE 0.492173728486323 Lambda1 -0.054955136
15 Train Loss 851.04565 Test MSE 851.7262002365724 Test RE 0.4912914137682947 Lambda1 -0.058792222
16 Train Loss 850.2268 Test MSE 850.489878930948 Test RE 0.4909347177916804 Lambda1 -0.060927033
17 Train Loss 848.6353 Test MSE 847.9781259384114 Test RE 0.4902092426520059 Lambda1 -0.062106717
18 Train Loss 847.26105 Test MSE 845.8439232118347 Test RE 0.48959197141812183 Lambda1 -0.06185923
19 Train Loss 843.8424 T

92 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
93 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
94 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
95 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
96 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
97 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
98 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
99 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
100 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
101 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
102 Trai

175 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
176 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
177 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
178 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
179 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
180 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
181 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
182 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
183 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06
184 Train Loss 270.08582 Test MSE 279.7043397237661 Test RE 0.2815391356595941 Lambda1 7.978391e-06


In [14]:
import scipy.io as sio

In [15]:
for tune_reps in range(25):
    label = "inv_HT_atanh_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
    re = np.array(data["test_re_loss"])
    print(tune_reps," ",np.mean(re[:,-1]))

FileNotFoundError: [Errno 2] No such file or directory: 'inv_HT_atanh_tune0.mat'

In [ ]:
lrn_tune[10]